In [1]:
import sys
sys.path.insert(0, '..')

In [2]:
from flood_forecast.deployment.inference import InferenceMode
from datetime import datetime
from typing import Dict, List
import os
import pandas as pd
from pandas import DataFrame, Series, DatetimeIndex
import wandb
import numpy as np

In [9]:
def make_config_file_infer(file_path: str, df_len: int) -> Dict:
    """sample of a config object for the inference case"""
    train_number: float = df_len * .7
    validation_number: float = df_len * .9
    config_default = {
        "model_name": "DecoderTransformer",
        "model_type": "PyTorch",
        "takes_target": False,
        "model_params": {
            "n_time_series": 30,
            "n_head": 8,
            "forecast_history": 120,
            "n_embd": 1,
            "num_layer": 5,
            "dropout": 0.1,
            "q_len": 1,
            "scale_att": False,
            "forecast_length": 30,
            "additional_params": {}
        },
        "dataset_params":
            {
                "class": "default",
                "training_path": file_path,
                "validation_path": file_path,
                "test_path": file_path,
                "batch_size": 64,
                "forecast_history": 120,
                "forecast_length": 30,
                "train_end": int(train_number),
                "valid_start": int(train_number + 1),
                "valid_end": int(validation_number),
                "target_col": ['Austria'],
                "relevant_cols": ['Austria', 'Belgium', 'Bulgaria', 'Switzerland', 'Czechia',
                                  'Germany', 'Denmark', 'Estonia', 'Spain', 'Finland', 'France', 'Greece',
                                  'Croatia', 'Hungary', 'Ireland', 'Italy', 'Lithuania', 'Luxembourg',
                                  'Latvia', 'Netherlands', 'Norway', 'Poland', 'Portugal', 'Romania',
                                  'Slovenia', 'Slovakia', 'Sweden', 'United Kingdom', 'month', 'weekday'],
                "scaler": "StandardScaler",
                "interpolate": False,
                "sort_column": "time",
            },
        "training_params":
            {
                "criterion": "DilateLoss",
                "optimizer": "Adam",
                "optim_params":
                    {
                    },
                "lr": 0.001,
                "epochs": 100,
                "batch_size": 64
            },
        "early_stopping": {
            "patience": 3
        },
        "GCS": False,
        "sweep": False,
        "wandb": False,
        "forward_params": {},
        "metrics": ["DilateLoss"],
        "inference_params":
            {
                "datetime_start": "2013-05-01",
                "hours_to_forecast": 30,
                "test_csv_path": file_path,
                "decoder_params": {
                    "decoder_function": "simple_decode",
                    "unsqueeze_dim": 1
                },
                "dataset_params": {
                    "file_path": file_path,
                    "forecast_history": 120,
                    "forecast_length": 30,
                    "relevant_cols": ['Austria', 'Belgium', 'Bulgaria', 'Switzerland', 'Czechia',
                                      'Germany', 'Denmark', 'Estonia', 'Spain', 'Finland', 'France', 'Greece',
                                      'Croatia', 'Hungary', 'Ireland', 'Italy', 'Lithuania', 'Luxembourg',
                                      'Latvia', 'Netherlands', 'Norway', 'Poland', 'Portugal', 'Romania',
                                      'Slovenia', 'Slovakia', 'Sweden', 'United Kingdom', 'month', 'weekday'],
                    "target_col": ['Austria'],
                    "scaling": "StandardScaler",
                    "interpolate_param": False
                }
            },
    }

    return config_default

In [10]:
file_path: str = '../data/wind_train.csv'
full_len: int = len(pd.read_csv(file_path))

In [11]:
ckpt_path: str = r'C:\Users\Lorenzo\PycharmProjects\flow-forecast\notebooks'

In [12]:
inf = InferenceMode(forecast_steps=18, num_prediction_samples=1, model_params=make_config_file_infer(file_path, full_len), csv_path=file_path, weight_path=os.path.join(ckpt_path, 'checkpoint.pth'), wandb_proj=None, torch_script=False)
df, tensor, history, forecast_start, test, samples = inf.infer_now(some_date=datetime(2013, 5, 3), csv_path=file_path, save_buck=None, save_name=None, use_torch_script=False)

Using model  <class 'flood_forecast.transformer_xl.transformer_bottleneck.DecoderTransformer'>
Weights sucessfully loaded
interpolate should be below
[]
Now loading ../data/wind_train.csv
scaling now
interpolate should be below
[]
Now loading ../data/wind_train.csv
scaling now
interpolate should be below
[]
Now loading ../data/wind_train.csv
scaling now
Using Wandb config:
<function PreInitCallable.<locals>.preinit_wrapper at 0x00000225C0146678>
Torch is using cuda
This model is currently forecasting for : 1 targets
interpolate should be below
[]
Now loading ../data/wind_train.csv
scaling now
CSV Path below
../data/wind_train.csv
Add debugging crap below


To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Add debugging crap below



A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [15]:
df_preds: DataFrame = df[['datetime', 'preds']]
df_preds.tail(18)

,datetime,preds
9984,2013-05-03,0.106555
9985,2013-05-04,0.188732
9986,2013-05-05,0.146243
9987,2013-05-06,0.161453
9988,2013-05-07,0.156739
9989,2013-05-08,0.139807
9990,2013-05-09,0.149072
9991,2013-05-10,0.152008
9992,2013-05-11,0.154331
9993,2013-05-12,0.108832
